# Flopy Tutorial on using time series in any package (chd, rch, etc).

From the flopy 3.3.5 readthedocs documentation

The idea here is to halp better understand the concept, which is very difficult from the origonal notebooks. Therefore, I added more text.

#TO 2023-12-15

In [35]:
# package import

# import os
# from pathlib import Path
from tempfile import TemporaryDirectory
import numpy as np
import flopy
from etc import attr

In [22]:
# set up where simulation workspace will be stored
temp_dir = TemporaryDirectory()
workspace = temp_dir.name
name = "tutorial03_mf6_data"

In [23]:
# create the flopy simulation and tdis objects
sim = flopy.mf6.MFSimulation(
    sim_name=name, exe_name="mf6", version="mf6", sim_ws=workspace
)

tdis_rc = [ (1.0, 1, 1.0),
           (10.0, 5, 1.0),
           (10.0, 5, 1.0),
           (10.0, 1, 1.0)] # PERLEN, NSTEP, TSMULT

tdis_package = flopy.mf6.modflow.mftdis.ModflowTdis(
                sim, time_units="DAYS",
                nper=len(tdis_rc),
                perioddata=tdis_rc
)

# create the Flopy groundwater flow (gwf) model object
gwf = flopy.mf6.ModflowGwf(sim, 
                           modelname=name,
                           model_nam_file=f"{name}.nam",
)

# create the flopy iterative model solver (ims) package object
ims = flopy.mf6.modflow.mfims.ModflowIms(sim,
                                         pname="ims",
                                         complexity="SIMPLE",
)

# create the discretization package
dis = flopy.mf6.modflow.mfgwfdis.ModflowGwfdis( gwf,
    pname="dis",
    nogrb=True,
    nlay=3,
    nrow=101,
    ncol=101,
    delr=4,
    delc=4,
    top=0.0,
    botm=np.linspace(-3.0, -50.0 / 3.0, 3),
)

# create the initial condition (ic)
ic_package = flopy.mf6.modflow.mfgwfic.ModflowGwfic(gwf, strt=50.0)

# create node property flow (npf) packages
npf_package = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf( gwf,
    save_flows=True,
    icelltype=[1, 0, 0],
    k=[5.0, 0.1, 4.0],
    k33=[0.5, 0.005, 0.1],
)


# Time Series Example 1

One way to construct a time series is to pass the time series dictionary to the parent package constructor.

This example uses time series data in a GHB package. First the GHB stress_period_data is built.

The stress period data lists the cells for thie package and its data or the name of the time series instead.

The stress period data with the name of the time series can not be linked to different stress periods as this
is done internally by the chosen interpolation method. For a transient model, therefore, the dict of stress period data
has only one entrance, i.e. zero, the first stress period.

In [24]:
# build ghb stress period data
ghb_spd_ts = {}
ghb_period = []

for layer, cond in zip(range(1, 3), [15.0, 1500.0]):
    for row in range(0, 15):
        ghb_period.append(((layer, row, 9), "tides", cond, "Estuary-L2")) # cellid, tsname, cond, boundaryname for ref.

# Due to using time series only the first stress period is used in the dictionary. If Transient, else just use the period data.
ghb_spd_ts[0] = ghb_period

## Build the time series

The time series data is constructed as a list of tuples, each containing a time and the value (or values) at that time.

The time series data is put in a dictionary along with additional time series information including filename, time_series_namerecord, interpolation_methodrecord, and sfacrecord.

In [29]:
# build ts data
ts_data = []
for n in range(0, 365):
    time = float(n / 11.73)
    val = float(n / 60.0)
    ts_data.append((time, val))

# TODO: Can this be an array?
ts_data = np.vstack((np.arange(365.) / 11.73,
                     np.arange(365.) / 60.0)).T
    
# Create the ts dictionary
ts_dict = {
    "filename": "tides.ts",
    "time_series_namerecord": "tide",
    "timeseries": ts_data,
    "interpolation_methodrecord": "linearend",
    "sfacrecord": 1.1,
}

## Build the GHB package

The GHB package is then constructed, passing the time series data into the timeseries parameter.


In [41]:
# build ghb package
ghb = flopy.mf6.modflow.mfgwfghb.ModflowGwfghb(
    gwf,
    print_input=True,
    print_flows=True,
    save_flows=True,
    boundnames=True,
    timeseries=ts_dict,  # <---- here goes the time series dictionary.
    pname="ghb",
    maxbound=30,
    stress_period_data=ghb_spd_ts, # <---- here go the stress period data.
)

In [42]:
# What's in ghb.ts?
attr(ghb.ts)

['append_package', 'initialize', 'package_abbr']

In [43]:
# set required time series attributes
ghb.ts.time_series_namerecord = "tides"

attr(ghb.ts)

['append_package', 'initialize', 'package_abbr']

In [50]:
ghb.ts.fiets='Koga'
ghb.ts.step='Gazelle'
print(attr(ghb.ts))   # New attributes do not show up.
print(ghb.ts.step)           # However, they have  een stored in the object.
ghb.ts.time_series_namerecord


['append_package', 'initialize', 'package_abbr']
Gazelle


{internal}
(rec.array([('tides',)],
          dtype=[('time_series_names', 'O')]))

# Time Series Example 2

Another way to construct a time series is to initialize the time series through the ghb.ts.initialize method.

Additional time series can then be appended using the append_package method.


## First the GHB stress period data is built.

In [51]:
# clean up for next example
gwf.remove_package("ghb")

A ghb package is constructed **without the time series data**

In [52]:
# build ghb package
ghb = flopy.mf6.modflow.mfgwfghb.ModflowGwfghb( gwf,
    print_input = True,
    print_flows = True,
    save_flows  = True,
    boundnames  = True,
    pname="ghb",
    maxbound=30, # is this kind of arbitrary ???
    stress_period_data=ghb_spd_ts,
)

The the time series data are generated or obtained in any way.

In [53]:
# build ghb stress period data

ghb_period = []

for layer, cond in zip(range(1, 3), [15.0, 1500.0]):
    for row in range(0, 15):
        if row < 10:
            ghb_period.append(((layer, row, 9), "tides", cond, "Estuary-L2"))
        else:
            ghb_period.append(((layer, row, 9), "wl", cond, "Estuary-L2"))
            
ghb_spd_ts = {0: ghb_period}  # if transient put in dict with key 0, else just set ghb_spd_ts = ghb_period

Next the time series data is built.


The time series data is constructed as a list of tuples, with each tuple containing a time and the value (or values) at that time.

Or, equivalently as a 2D array where the first column is time and the second the data.

In [54]:
# build ts data
ts_data = []

for n in range(0, 365):
    time = float(n / 11.73)
    val = float(n / 60.0)
    ts_data.append((time, val))

ts_data2 = []
for n in range(0, 365):
    time = float(1.0 + (n / 12.01))
    val = float(n / 60.0)
    ts_data2.append((time, val))

ts_data3 = []
for n in range(0, 365):
    time = float(10.0 + (n / 12.01))
    val = float(n / 60.0)
    ts_data3.append((time, val))
    
# Or as ndarrays
ts_data1 = np.vstack((np.arange(365.) / 11.73, np.arange(365.) / 60.0)).T
ts_data2 = np.vstack((np.arange(365.) / 12.01, np.arange(365.) / 60.0)).T
ts_data3 = np.vstack((np.arange(365.) / 12.01, np.arange(365.) / 60.0)).T

The first time series data are added by calling the **initialize** method from the ghb.ts object.

Note that the time series data are just time series with addictioinal records for meta info regarding 
the series and pertaining to the packages to which they are connected. Many time series can be connected
to every package. Even the same time series could be connected to different packages. They will be
referenced in th stress period data by their name. The data itself is refferred to by the file name, as
a file will be written for each time series.

In [59]:
# initialize first time series
ghb.ts.initialize(
    filename="tides.ts",
    timeseries=ts_data1,
    time_series_namerecord="tides",
    interpolation_methodrecord="linearend",
    sfacrecord=1.1,
)

In [60]:
# append additional time series
ghb.ts.append_package(
    filename="wls.ts",
    timeseries=ts_data2,
    time_series_namerecord="wl",
    interpolation_methodrecord="linear", # Can be any string, checking is done only durung model run.
    sfacrecord=1.2,
)
# append additional time series
ghb.ts.append_package(
    filename="wls2.ts",
    timeseries=ts_data3,
    time_series_namerecord="wl2",
    interpolation_methodrecord="stepwise",
    sfacrecord=1.3,
)

Information can be retrieved from time series packages using the ts attribute of its parent package. Below the interpolation method record for each of the three time series are retrieved.

In [61]:
print(
    "{} is using {} interpolation".format(
        ghb.ts[0].filename,
        ghb.ts[0].interpolation_methodrecord.get_data()[0][0],
    )
)
print(
    "{} is using {} interpolation".format(
        ghb.ts[1].filename,
        ghb.ts[1].interpolation_methodrecord.get_data()[0][0],
    )
)
print(
    "{} is using {} interpolation".format(
        ghb.ts[2].filename,
        ghb.ts[2].interpolation_methodrecord.get_data()[0][0],
    )
)

tides.ts is using linearend interpolation
wls.ts is using koga interpolation
wls2.ts is using stepwise interpolation


In [58]:
try:
    temp_dir.cleanup()
except PermissionError:
    # can occur on windows: https://docs.python.org/3/library/tempfile.html#tempfile.TemporaryDirectory
    pass
